# Processing of a list of images

Select a result folder with the list of image files to process.

In [2]:
import yaml
from pathlib import Path
from ipyfilechooser import FileChooser

srcdir, dstdir = "", ""
if Path("config.yml").exists():
    with open("config.yml", "r") as file:
        config = yaml.safe_load(file)
        if "source" in config.keys():
            srcdir = Path(config["source"])
        if "destination" in config.keys():
            dstdir = Path(config["destination"])

fc = FileChooser(dstdir, select_desc="Destination")
display(fc)

FileChooser(path='/home/jeromeb/work/code/octopus/results', filename='', title='', show_hidden=False, select_d…

Load the list of files.

In [3]:
import pandas as pd

dstdir = Path(fc.selected) if fc.selected is not None else Path(dstdir)
filelistname = dstdir / "filelist.csv"
print("Destination folder :", dstdir)
print("Output file list :", filelistname)
filelist = pd.read_csv(filelistname)
filelist

Destination folder : /home/jeromeb/work/code/octopus/results
Output file list : /home/jeromeb/work/code/octopus/results/filelist.csv


,folder,name,channel1,channel2,channel3,channel4
0,/media/cephfs/acourtney/HCR_Octopus_Jerome/Feb...,2023_13Dec_4a17Dec_F2_Stitched_Cropped_ForAnal...,vglut_B4_546,ppp1_B1_647,Th_B3_488,Nuclei
1,/media/cephfs/acourtney/HCR_Octopus_Jerome/Feb...,2023_12Dec_2a17Dec_F2_Stitched_Cropped_ForAnal...,tbh_B4_546,LGC007_B3_647,vacht_B2_488,Nuclei
2,/media/cephfs/acourtney/HCR_Octopus_Jerome/Feb...,2023_20Dec_2aBatch2_12Dec_F1_Stitched_Cropped_...,LGC041_B2_546,LGC046_B2_647,Th_B3_488,Nuclei
3,/media/cephfs/acourtney/HCR_Octopus_Jerome/Feb...,14Aug2023_3b_Slices_F1_Stitched_Cropped_ForAna...,LGC009_B1_647,vglut_B4_546,Th_B3_488,Nuclei


In [33]:
row = filelist.iloc[0]
channel = int(row[[x.lower() == 'nuclei' for x in row]].index.item().replace('channel',''))-1
nchannels = 4
print(channel)
[ k for k  in range(nchannels) if k != channel ]


3


[0, 1, 2]

In [39]:
{'a':1, **{f'c{k}':k for k in range(4)}}

{'a': 1, 'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3}

Define processing functions

In [7]:
import numpy as np
import numpy.ma as ma
import dask.array as da
from skimage import measure
import pandas as pd
import json
from imaris_ims_file_reader.ims import ims
import tifffile
import napari
from scipy import ndimage as ndi


def get_files(dstdir, row, key=None):
    if key == "ims":
        return Path(row["folder"]) / row["name"]
    elif key == "regions":
        return Path(dstdir / str(row["name"]).replace(".ims", "-regions.json"))
    elif key == "labels":
        return Path(dstdir / str(row["name"]).replace(".ims", "-labels.tif"))
    elif key == "measurements":
        return Path(dstdir / str(row["name"]).replace(".ims", "-measurements.csv"))
    else:
        return {
            "ims": get_files(dstdir, row, "ims"),
            "regions": get_files(dstdir, row, "regions"),
            "labels": get_files(dstdir, row, "labels"),
            "measurements": get_files(dstdir, row, "measurements"),
        }


def load_regions_as_labels(dstdir: Path, row: pd.Series, shape, resolution_level: int):
    """Load regions as labels

    Parameters
    ----------
    dstdir: pathlib.Path
    row : pandas.Series
    shape : List
    resolution_level: int

    Result
    ------
    rois: numnpy.ndarray
        labels image
    """

    with open(get_files(dstdir, row, "regions"), "r") as outfile:
        poly = json.load(outfile)

    c = pow(2, poly["resolution_level"] - resolution_level)
    sdata = [c * np.array(p) for p in poly["regions"]]
    rois2d = napari.layers.Shapes(sdata, shape_type="polygon").to_labels(
        labels_shape=[shape[1], shape[2]]
    )
    rois = np.zeros(shape)
    for k in range(shape[0]):
        rois[k, 0 : rois2d.shape[0], 0 : rois2d.shape[1]] = rois2d
    return rois


def preprocess(img):
    """Preprocess the image before FISH intensity measurement

    Parameter
    ---------
    img : numpy.ndarray
        input image
    Result
    ------
    numpy.ndarray
        result
    """
    img = img.astype(np.float32)
    d = ndi.gaussian_filter(img, 1) - ndi.gaussian_filter(img, 2)
    t = 1.48 * np.median(np.abs(d - np.median(d)))
    return np.maximum(0, d / t - 4)


def preprocess(img):
    """Preprocess the image before FISH intensity measurement

    Parameter
    ---------
    img : numpy.ndarray
        input image
    Result
    ------
    numpy.ndarray
        result
    """
    img = img.astype(np.float32)
    # smooth tiles
    tmp = img.copy()
    tmp[tmp == 0] = np.mean(tmp[tmp > 0])
    for _ in range(7):
        tmp = ndi.gaussian_filter(tmp, [2, 2, 2])
        tmp[img > 0] = img[img > 0]
    # difference of Gaussians
    d = ndi.gaussian_filter(tmp, [0.5, 1, 1]) - ndi.gaussian_filter(tmp, [1, 2, 2])
    # relu with 4 std
    t = 1.48 * np.median(np.abs(d - np.median(d)))
    return np.maximum(0, d / t - 4)


def record_intensity(labels, score, rois, channels):
    z, y, x = np.meshgrid(*[np.arange(n) for n in labels.shape])
    dst = []
    label_list = np.unique(labels)
    for idx in label_list[1:]:
        mask = labels != idx
        dst.append(
            {
                "label": idx,
                "roi": ma.MaskedArray(rois, mask).min(),
                "x": ma.MaskedArray(x, mask).mean(),
                "y": ma.MaskedArray(y, mask).mean(),
                "z": ma.MaskedArray(z, mask).mean(),
                "area": np.logical_not(mask).sum(),
                **{f'c{channels[k]}': ma.MaskedArray(score[0], mask).mean() for k in range(score.shape[0])},                
            }
        )
    return pd.DataFrame.from_records(dst)


def process_item(row, resolution_level: int, model, dstdir: Path):
    """Process a line of the input filelist

    Segment the nuclei and measure the FISH signal
    """
    
    # identify the channel with nuclar label
    nuclear_channel = int(row[[x.lower() == 'nuclei' for x in row]].index.item().replace('channel','')) - 1

    # load the image
    store = ims(
        get_files(dstdir, row, "ims"), ResolutionLevelLock=resolution_level, aszarr=True
    )
    img = da.from_zarr(store)

    nchannels = img.shape[1]

    # segment the cell
    labels = model.eval(
        img[:, nuclear_channel].compute().squeeze(),
        diameter=30 / resolution_level,
        do_3D=False,
        anisotropy=3,
        min_size=500,
        stitch_threshold=0.1,
    )[0]

    tifffile.imwrite(get_files(dstdir, row, "labels"), labels)

    # load the regions
    rois = load_regions_as_labels(dstdir, row, labels.shape, resolution_level)
    
    # preprocess the channels    
    channels = [k for k in range(nchannels) if k != nuclear_channel]
    score = np.stack(
        [
            preprocess(img[0, k].compute().squeeze().astype(np.float32))
            for k in range(nchannels) if k != nuclear_channel
        ],
        0,
    )

    df = record_intensity(labels, score, rois, channels)

    df.to_csv(get_files(dstdir, row, "measurements"), index=False)

    return df

Process all files

In [8]:
from cellpose import models, core

model = models.Cellpose(gpu=core.use_gpu(), model_type="nuclei")
resolution_level = 1

for row in filelist.iloc:
    if get_files(dstdir, row, "measurements").exists() is False:
        if get_files(dstdir, row, "regions").exists() is True:
            print(row)
            process_item(row, resolution_level, model, dstdir)

Unnamed: 0                                                    1
folder        /media/cephfs/acourtney/HCR_Octopus_Jerome/Feb...
name          2023_12Dec_2a17Dec_F2_Stitched_Cropped_ForAnal...
Name: 1, dtype: object
Opening readonly file: /media/cephfs/acourtney/HCR_Octopus_Jerome/Febuary2024/2023_12Dec_2a17Dec_F2_Stitched_Cropped_ForAnalysis.ims 

GET : .zarray
Unnamed: 0                                                    3
folder        /media/cephfs/acourtney/HCR_Octopus_Jerome/Feb...
name          14Aug2023_3b_Slices_F1_Stitched_Cropped_ForAna...
Name: 3, dtype: object
Opening readonly file: /media/cephfs/acourtney/HCR_Octopus_Jerome/Febuary2024/14Aug2023_3b_Slices_F1_Stitched_Cropped_ForAnalysis.ims 

GET : .zarray
Closing file: /media/cephfs/acourtney/HCR_Octopus_Jerome/Febuary2024/14Aug2023_3b_Slices_F1_Stitched_Cropped_ForAnalysis.ims 

Closing file: /media/cephfs/acourtney/HCR_Octopus_Jerome/Febuary2024/2023_12Dec_2a17Dec_F2_Stitched_Cropped_ForAnalysis.ims 

